In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [3]:
dataset = pd.read_csv('dataset.csv')
dataset

,id,eye,stimuli,color,glaucoma,sample_1,sample_2,sample_3,sample_4,sample_5,...,sample_2761,sample_2762,sample_2763,sample_2764,sample_2765,sample_2766,sample_2767,sample_2768,sample_2769,sample_2770
0,C001,left,ramp,white,no,11.421809,51.632774,49.136442,44.595915,10.767278,...,55.015610,55.195877,55.131669,55.220485,54.954781,55.044311,54.868641,55.012999,55.104108,55.182648
1,C001,left,ramp,blue,no,56.805336,56.765305,56.750474,56.175521,56.930798,...,27.476767,52.824345,53.477177,55.411936,56.438331,56.553257,56.073525,56.063473,55.947377,55.893032
2,C001,left,ramp,green,no,54.326952,54.440178,54.714699,55.499070,56.050834,...,56.577864,56.609074,56.819729,56.770057,56.185188,13.059161,17.648903,55.145064,14.972549,56.430405
3,C001,left,flash,white,no,55.787624,55.933990,55.942581,56.724411,56.979560,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C001,left,flash,blue,no,59.850759,57.988526,33.327293,58.638396,57.587477,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,S020,right,flash,red,yes,46.334814,46.699833,46.876923,46.748890,47.495081,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
652,S020,right,ramp,red,yes,53.715813,54.285419,53.435158,53.306038,45.591537,...,50.756836,50.126942,50.812572,50.563976,50.741753,50.015706,49.969231,49.709682,49.624617,49.315861
653,S020,right,flash,white,yes,51.445000,50.907558,52.043118,51.082579,51.404472,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
654,S020,right,ramp,green,yes,0.000000,0.000000,0.000000,0.000000,0.000000,...,47.576739,47.825294,47.877976,47.534389,47.591788,48.315285,47.707390,48.059217,47.963902,47.869469


In [13]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.fftpack import fft

def plot_signal_and_fourier(dataset, index, sampling_rate=29.84):
    """
    Muestra un gráfico de la señal en el tiempo y su transformada de Fourier (frecuencia) para un índice específico,
    además de mostrar la información del paciente.

    Parameters:
    - dataset: El DataFrame que contiene la señal.
    - index: El índice de la fila que se desea graficar.
    - sampling_rate: La frecuencia de muestreo en Hz (muestras por segundo).
    """
    # Verificar que el índice esté dentro del rango del dataset
    if index < 0 or index >= len(dataset):
        raise IndexError("Índice fuera de rango del dataset.")

    # Extraer la información de la fila correspondiente al índice
    patient_info = dataset.loc[index, ['id', 'eye', 'stimuli', 'color', 'glaucoma']]
    samples = dataset.loc[index, [f'sample_{i+1}' for i in range(2770)]].values

    # Asegurar que los datos sean de tipo float, convirtiendo valores no numéricos a NaN
    samples = pd.to_numeric(samples, errors='coerce')

    # Mostrar información del paciente
    print("Información del paciente:")
    print(f"ID: {patient_info['id']}")
    print(f"Ojo: {patient_info['eye']}")
    print(f"Estímulo: {patient_info['stimuli']}")
    print(f"Color: {patient_info['color']}")
    print(f"Glaucoma: {patient_info['glaucoma']}")
    
    # Eliminar los valores NaN de la señal antes de hacer la FFT
    samples_no_nan = samples[~np.isnan(samples)]
    
    # --- Señal en el tiempo ---
    time = np.arange(0, len(samples_no_nan)) / sampling_rate  # Tiempo en segundos
    
    fig_time = go.Figure()
    fig_time.add_trace(go.Scatter(
        x=time, 
        y=samples_no_nan, 
        mode='lines', 
        name='Señal en el tiempo',
        hovertemplate='Tiempo: %{x:.2f} s<br>Valor: %{y:.2f}'
    ))
    
    fig_time.update_layout(
        title=f'Señal en el tiempo - Índice {index}',
        xaxis_title='Tiempo (s)',
        yaxis_title='Valor de la señal',
        template='plotly_white'
    )
    
    # Mostrar el gráfico de la señal en el tiempo
    fig_time.show()
    
    # --- Transformada de Fourier ---
    # Calcular la Transformada de Fourier solo con los valores sin NaN
    N = len(samples_no_nan)
    if N > 0:  # Asegurarse de que hay muestras válidas
        yf = fft(samples_no_nan)
        xf = np.fft.fftfreq(N, d=1 / sampling_rate)[:N // 2]  # Frecuencia con la tasa de muestreo
        
        # Gráfico de la transformada de Fourier
        fig_fourier = go.Figure()
        fig_fourier.add_trace(go.Scatter(
            x=xf, 
            y=2.0 / N * np.abs(yf[:N // 2]), 
            mode='lines', 
            name='Señal en frecuencia',
            hovertemplate='Frecuencia: %{x:.2f} Hz<br>Valor: %{y:.2f}'
        ))
        
        fig_fourier.update_layout(
            title=f'Transformada de Fourier - Índice {index}',
            xaxis_title='Frecuencia (Hz)',
            yaxis_title='Magnitud',
            template='plotly_white'
        )
        
        # Mostrar el gráfico de la señal en frecuencia
        fig_fourier.show()
    else:
        print(f"No hay suficientes datos para calcular la FFT en el índice {index}.")


In [15]:
plot_signal_and_fourier(dataset, 200, sampling_rate=29.84)

Información del paciente:
ID: C013
Ojo: right
Estímulo: ramp
Color: green
Glaucoma: no


In [33]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.fftpack import fft
from scipy.signal import butter, filtfilt

def butter_lowpass_filter(data, cutoff, fs, order=5):
    """
    Aplica un filtro de paso bajo a los datos.
    
    Parameters:
    - data: Los datos de la señal.
    - cutoff: La frecuencia de corte (Hz).
    - fs: La frecuencia de muestreo (Hz).
    - order: El orden del filtro (cuanto más alto, más pronunciado será el filtrado).
    
    Returns:
    - La señal filtrada.
    """
    nyquist = 0.5 * fs  # Frecuencia de Nyquist
    normal_cutoff = cutoff / nyquist  # Frecuencia de corte normalizada
    # Diseñar el filtro
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    # Aplicar el filtro
    y = filtfilt(b, a, data)
    return y

def plot_signal_and_fourier_filtered(dataset, index, sampling_rate=29.84, cutoff_freq=2, order=5):
    """
    Muestra un gráfico de la señal en el tiempo y su transformada de Fourier (frecuencia),
    además de aplicar un filtro de paso bajo para reducir el ruido.

    Parameters:
    - dataset: El DataFrame que contiene la señal.
    - index: El índice de la fila que se desea graficar.
    - sampling_rate: La frecuencia de muestreo en Hz (muestras por segundo).
    - cutoff_freq: La frecuencia de corte para el filtro de paso bajo.
    """
    # Verificar que el índice esté dentro del rango del dataset
    if index < 0 or index >= len(dataset):
        raise IndexError("Índice fuera de rango del dataset.")

    # Extraer la información de la fila correspondiente al índice
    patient_info = dataset.loc[index, ['id', 'eye', 'stimuli', 'color', 'glaucoma']]
    samples = dataset.loc[index, [f'sample_{i+1}' for i in range(2770)]].values

    # Asegurar que los datos sean de tipo float, convirtiendo valores no numéricos a NaN
    samples = pd.to_numeric(samples, errors='coerce')

    # Eliminar los valores NaN de la señal antes de hacer la FFT
    samples_no_nan = samples[~np.isnan(samples)]
    
    # Aplicar el filtro de paso bajo
    filtered_signal = butter_lowpass_filter(samples_no_nan, cutoff=cutoff_freq, fs=sampling_rate, order=order)
    
    # Mostrar información del paciente
    print("Información del paciente:")
    print(f"ID: {patient_info['id']}")
    print(f"Ojo: {patient_info['eye']}")
    print(f"Estímulo: {patient_info['stimuli']}")
    print(f"Color: {patient_info['color']}")
    print(f"Glaucoma: {patient_info['glaucoma']}")
    
    # --- Señal en el tiempo (filtrada) ---
    time = np.arange(0, len(filtered_signal)) / sampling_rate  # Tiempo en segundos
    
    fig_time = go.Figure()
    fig_time.add_trace(go.Scatter(
        x=time, 
        y=filtered_signal, 
        mode='lines', 
        name='Señal filtrada en el tiempo',
        hovertemplate='Tiempo: %{x:.2f} s<br>Valor: %{y:.2f}'
    ))
    
    fig_time.update_layout(
        title=f'Señal filtrada en el tiempo - Índice {index}',
        xaxis_title='Tiempo (s)',
        yaxis_title='Valor de la señal',
        template='plotly_white'
    )
    
    # Mostrar el gráfico de la señal en el tiempo
    fig_time.show()
    
    # --- Transformada de Fourier ---
    # Calcular la Transformada de Fourier solo con los valores sin NaN
    N = len(filtered_signal)
    if N > 0:  # Asegurarse de que hay muestras válidas
        yf = fft(filtered_signal)
        xf = np.fft.fftfreq(N, d=1 / sampling_rate)[:N // 2]  # Frecuencia con la tasa de muestreo
        
        # Gráfico de la transformada de Fourier
        fig_fourier = go.Figure()
        fig_fourier.add_trace(go.Scatter(
            x=xf, 
            y=2.0 / N * np.abs(yf[:N // 2]), 
            mode='lines', 
            name='Señal en frecuencia',
            hovertemplate='Frecuencia: %{x:.2f} Hz<br>Valor: %{y:.2f}'
        ))
        
        fig_fourier.update_layout(
            title=f'Transformada de Fourier - Índice {index}',
            xaxis_title='Frecuencia (Hz)',
            yaxis_title='Magnitud',
            template='plotly_white'
        )
        
        # Mostrar el gráfico de la señal en frecuencia
        fig_fourier.show()
    else:
        print(f"No hay suficientes datos para calcular la FFT en el índice {index}.")


In [26]:
plot_signal_and_fourier(dataset, 0, sampling_rate=29.84)

Información del paciente:
ID: C001
Ojo: left
Estímulo: ramp
Color: white
Glaucoma: no


In [47]:
plot_signal_and_fourier_filtered(dataset, 0, sampling_rate=29.84, cutoff_freq=2, order=5)

Información del paciente:
ID: C001
Ojo: left
Estímulo: ramp
Color: white
Glaucoma: no
